In [3]:
import pandas as pd
import numpy as np

In [4]:
df=pd.read_csv('solar.csv')
df

,Catalog Number,Calendar Date,Eclipse Time,Delta T (s),Lunation Number,Saros Number,Eclipse Type,Gamma,Eclipse Magnitude,Latitude,Longitude,Sun Altitude,Sun Azimuth,Path Width (km),Central Duration
0,1,-1999 June 12,03:14:51,46438,-49456,5,T,-0.2701,1.0733,6.0N,33.3W,74,344,247,06m37s
1,2,-1999 December 5,23:45:23,46426,-49450,10,A,-0.2317,0.9382,32.9S,10.8E,76,21,236,06m44s
2,3,-1998 June 1,18:09:16,46415,-49444,15,T,0.4994,1.0284,46.2N,83.4E,60,151,111,02m15s
3,4,-1998 November 25,05:57:03,46403,-49438,20,A,-0.9045,0.9806,67.8S,143.8W,25,74,162,01m14s
4,5,-1997 April 22,13:19:56,46393,-49433,-13,P,-1.4670,0.1611,60.6S,106.4W,0,281,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11893,11894,2998 December 10,03:18:31,4414,12355,187,P,1.2838,0.4773,67.2N,145.0E,0,179,NaN,NaN
11894,11895,2999 May 6,23:23:57,4417,12360,154,T,0.8388,1.0566,71.5N,177.3E,33,146,345,03m25s
11895,11896,2999 October 30,09:34:33,4420,12366,159,A-,-1.0023,0.9586,70.9S,84.7W,0,137,-,-
11896,11897,3000 April 26,14:18:06,4424,12372,164,T,0.1310,1.0222,21.1N,18.4W,82,166,76,02m11s


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11898 entries, 0 to 11897
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Catalog Number     11898 non-null  int64  
 1   Calendar Date      11898 non-null  object 
 2   Eclipse Time       11898 non-null  object 
 3   Delta T (s)        11898 non-null  int64  
 4   Lunation Number    11898 non-null  int64  
 5   Saros Number       11898 non-null  int64  
 6   Eclipse Type       11898 non-null  object 
 7   Gamma              11898 non-null  float64
 8   Eclipse Magnitude  11898 non-null  float64
 9   Latitude           11898 non-null  object 
 10  Longitude          11898 non-null  object 
 11  Sun Altitude       11898 non-null  int64  
 12  Sun Azimuth        11898 non-null  int64  
 13  Path Width (km)    7698 non-null   object 
 14  Central Duration   7698 non-null   object 
dtypes: float64(2), int64(6), object(7)
memory usage: 1.4+ MB


In [5]:
df.isnull().sum()

Catalog Number          0
Calendar Date           0
Eclipse Time            0
Delta T (s)             0
Lunation Number         0
Saros Number            0
Eclipse Type            0
Gamma                   0
Eclipse Magnitude       0
Latitude                0
Longitude               0
Sun Altitude            0
Sun Azimuth             0
Path Width (km)      4381
Central Duration     4294
dtype: int64

In [6]:
from datetime import datetime

# Step 1: Flag Common or Current Era (CE) dates (بعد الميلاد)
df['Is_CE'] = ~df['Calendar Date'].str.startswith('-')

# Step 2: Strip and clean whitespace for reading dates later on
df['Calendar Date Clean'] = df['Calendar Date'].str.strip().str.replace(r'\s+', ' ', regex=True)

# Step 3: Remove leading '-' from BCE dates for parsing (so we can make it datetime)
df['Calendar Date Clean'] = df['Calendar Date Clean'].str.lstrip('-')

# Step 4: Pad year with zeros to ensure four-digit format as strptime accepts range 1 and upove in this format only
def pad_year(date_str):
    parts = date_str.split(" ", 1)
    if len(parts) > 1 and len(parts[0]) < 4:
        parts[0] = parts[0].zfill(4)
    return " ".join(parts)

df['Calendar Date Clean'] = df['Calendar Date Clean'].apply(pad_year)

# Step 5: Safely parse the date using datetime (strptime) as to_datetime has range limitations (befre 1678 and after 2262 ) will not be shown in it
def safe_to_datetime(date_str, format='%Y %B %d'):
    try:
        return datetime.strptime(date_str, format)
    except Exception:
        return None

df['Calendar Date Parsed'] = df['Calendar Date Clean'].apply(safe_to_datetime)

# Final check (the 3 instances that werent shown bc strptime range from 1 and these instance are year 0)
valid_count = df['Calendar Date Parsed'].notna().sum()
print(f"✅ Valid parsed dates: {valid_count}")

df


✅ Valid parsed dates: 11895


,Catalog Number,Calendar Date,Eclipse Time,Delta T (s),Lunation Number,Saros Number,Eclipse Type,Gamma,Eclipse Magnitude,Latitude,Longitude,Sun Altitude,Sun Azimuth,Path Width (km),Central Duration,Is_CE,Calendar Date Clean,Calendar Date Parsed
0,1,-1999 June 12,03:14:51,46438,-49456,5,T,-0.2701,1.0733,6.0N,33.3W,74,344,247,06m37s,False,1999 June 12,1999-06-12 00:00:00
1,2,-1999 December 5,23:45:23,46426,-49450,10,A,-0.2317,0.9382,32.9S,10.8E,76,21,236,06m44s,False,1999 December 5,1999-12-05 00:00:00
2,3,-1998 June 1,18:09:16,46415,-49444,15,T,0.4994,1.0284,46.2N,83.4E,60,151,111,02m15s,False,1998 June 1,1998-06-01 00:00:00
3,4,-1998 November 25,05:57:03,46403,-49438,20,A,-0.9045,0.9806,67.8S,143.8W,25,74,162,01m14s,False,1998 November 25,1998-11-25 00:00:00
4,5,-1997 April 22,13:19:56,46393,-49433,-13,P,-1.4670,0.1611,60.6S,106.4W,0,281,NaN,NaN,False,1997 April 22,1997-04-22 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11893,11894,2998 December 10,03:18:31,4414,12355,187,P,1.2838,0.4773,67.2N,145.0E,0,179,NaN,NaN,True,2998 December 10,2998-12-10 00:00:00
11894,11895,2999 May 6,23:23:57,4417,12360,154,T,0.8388,1.0566,71.5N,177.3E,33,146,345,03m25s,True,2999 May 6,2999-05-06 00:00:00
11895,11896,2999 October 30,09:34:33,4420,12366,159,A-,-1.0023,0.9586,70.9S,84.7W,0,137,-,-,True,2999 October 30,2999-10-30 00:00:00
11896,11897,3000 April 26,14:18:06,4424,12372,164,T,0.1310,1.0222,21.1N,18.4W,82,166,76,02m11s,True,3000 April 26,3000-04-26 00:00:00


In [ ]:
df.to_csv('cleaned_dates.csv', index=False)

In [ ]:
isinstance(df['Calendar Date Parsed'][0], datetime)  

True